In [1]:
!pip install fastapi
!pip install uvicorn[standard]
!pip install python-multipart
!pip install chromadb
!pip install sentence-transformers
!pip install tqdm
!pip install PyMuPDF
!pip install pillow
!pip install pytesseract
!pip install sqlalchemy
!pip install aiofiles
!pip install python-dotenv
!pip install transformers accelerate sentencepiece
!pip install nest_asyncio
!pip install pyngrok




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 6.7 MB/s eta 0

# Ngrok setup

In [2]:
from getpass import getpass
import os

# This will NOT show your token when you type it
token = getpass("Paste your ngrok token (input hidden): ")

# Store it in an env var for later use
os.environ["NGROK_AUTH_TOKEN"] = token

print("Token stored in environment (not printed).")



Paste your ngrok token (input hidden): ··········
Token stored in environment (not printed).


In [3]:
from pyngrok import ngrok
import os

NGROK_TOKEN = os.environ.get("NGROK_AUTH_TOKEN")

if not NGROK_TOKEN:
    raise RuntimeError("Missing NGROK_AUTH_TOKEN env var. Did you run the getpass cell?")

ngrok.set_auth_token(NGROK_TOKEN)
print("ngrok token loaded from environment.")


ngrok token loaded from environment.


In [4]:
import os
print("Loaded secret:", bool(os.environ.get("NGROK_AUTH_TOKEN")))


Loaded secret: True


In [5]:
!rm -rf chroma_db


# RAG Pipeline

In [6]:
%%writefile main.py
"""
main.py
Multi-Modal RAG over PDF documents using:
- HuggingFace CLIP for unified multi-modal embeddings (text + images)
- HuggingFace FLAN-T5 for generation
- ChromaDB (new API, PersistentClient) as vector store
- FastAPI as API layer
"""

import io
import os
import re
import uuid
from typing import List, Dict, Optional

import fitz  # PyMuPDF
from PIL import Image
import pytesseract

import numpy as np
import torch
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModelForSeq2SeqLM

import chromadb
from chromadb import PersistentClient

# Configuration

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CLIP_MODEL_NAME = "openai/clip-vit-base-patch32"
GEN_MODEL_NAME = "google/flan-t5-base"  # can change to -small if GPU RAM is low

CHROMA_DIR = "./chroma_db"
COLLECTION_NAME = "multi_modal_docs"
IMAGE_DIR = "./pdf_images"
os.makedirs(IMAGE_DIR, exist_ok=True)



# Model loader (lazy singletons)

class ModelRegistry:
    _clip_model = None
    _clip_processor = None
    _tokenizer = None
    _gen_model = None

    @classmethod
    def get_clip(cls):
        if cls._clip_model is None or cls._clip_processor is None:
            print("Loading CLIP model:", CLIP_MODEL_NAME)
            cls._clip_model = CLIPModel.from_pretrained(CLIP_MODEL_NAME).to(DEVICE)
            cls._clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME)
        return cls._clip_model, cls._clip_processor

    @classmethod
    def get_generator(cls):
        if cls._tokenizer is None or cls._gen_model is None:
            print("Loading generator model:", GEN_MODEL_NAME)
            cls._tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_NAME)
            cls._gen_model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL_NAME).to(DEVICE)
        return cls._tokenizer, cls._gen_model



# Chroma client & collection (NEW API)


client: PersistentClient = chromadb.PersistentClient(path=CHROMA_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)



# Embedding helpers (CLIP)


def encode_text_clip(texts: List[str]) -> np.ndarray:
    """Encode a list of texts into CLIP text embeddings."""
    clip_model, clip_processor = ModelRegistry.get_clip()
    inputs = clip_processor(text=texts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    with torch.no_grad():
        text_embs = clip_model.get_text_features(**inputs)
    text_embs = text_embs / text_embs.norm(dim=-1, keepdim=True)
    return text_embs.cpu().numpy()


def encode_images_clip(image_paths: List[str]) -> np.ndarray:
    """Encode a list of image file paths into CLIP image embeddings."""
    clip_model, clip_processor = ModelRegistry.get_clip()
    images = [Image.open(p).convert("RGB") for p in image_paths]
    inputs = clip_processor(images=images, return_tensors="pt", padding=True).to(DEVICE)
    with torch.no_grad():
        img_embs = clip_model.get_image_features(**inputs)
    img_embs = img_embs / img_embs.norm(dim=-1, keepdim=True)
    return img_embs.cpu().numpy()



# PDF ingestion: text + images + OCR + simple captions


def extract_page_text(page) -> str:
    """Simple full-page text extraction."""
    return page.get_text("text")


def find_chart_captions(page_text: str) -> List[str]:
    """Find lines that look like figure/chart captions."""
    captions = []
    for line in page_text.splitlines():
        if re.match(r"^\s*(Figure|Chart|Exhibit)\b", line.strip(), flags=re.IGNORECASE):
            captions.append(line.strip())
    return captions


def ingest_pdf_bytes(doc_id: str, file_bytes: bytes) -> List[Dict]:
    """
    Ingest a single PDF from bytes.
    Returns a list of 'items', each with modality and metadata BEFORE chunking/embedding.
    """
    items: List[Dict] = []
    doc = fitz.open(stream=file_bytes, filetype="pdf")

    for page_index in range(len(doc)):
        page = doc[page_index]
        page_number = page_index + 1
        page_text = extract_page_text(page)
        page_text = page_text.strip()

        section_title = f"Page {page_number}"  # simple section label

        # ---- TEXT ITEM ----
        if page_text:
            items.append({
                "id": f"{doc_id}_text_p{page_number}",
                "doc_id": doc_id,
                "modality": "text",
                "content": page_text,
                "page_number": page_number,
                "section": section_title,
                "extra": {},
            })

        # ---- IMAGE ITEMS ----
        image_list = page.get_images(full=True)
        captions = find_chart_captions(page_text)
        caption_str = " | ".join(captions) if captions else ""

        for img_idx, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            pil_img = Image.open(io.BytesIO(image_bytes)).convert("RGB")

            img_filename = f"{doc_id}_page{page_number}_img{img_idx}.png"
            img_path = os.path.join(IMAGE_DIR, img_filename)
            pil_img.save(img_path)

            ocr_text = pytesseract.image_to_string(pil_img)

            items.append({
                "id": f"{doc_id}_img_p{page_number}_{img_idx}",
                "doc_id": doc_id,
                "modality": "image",
                "content": ocr_text.strip(),  # used in LLM context
                "page_number": page_number,
                "section": section_title,
                "extra": {
                    "image_path": img_path,
                    "caption": caption_str,
                },
            })

    doc.close()
    return items



# Smart Hybrid Chunking (Semantic + Structural)


def structural_split(text: str) -> List[str]:
    """
    Detect structural boundaries (section headers, numbered headings, all caps, etc.).
    Returns a list of larger blocks split by structure.
    """
    lines = text.split("\n")
    chunks = []
    buffer = []

    for line in lines:
        stripped = line.strip()
        if not stripped:
            buffer.append(line)
            continue

        is_heading = (
            re.match(r"^\d+(\.\d+)*\s", stripped) or         # 1.  / 1.1 / 2.3.4
            re.match(r"^[A-Z][A-Z\s\-]{6,}$", stripped) or   # ALL CAPS headings
            re.match(r"^[A-Za-z].+:\s*$", stripped) or       # Title:
            re.match(r"^(Conclusion|Summary|Overview)\b", stripped, re.I)
        )

        # New section detected → flush buffer
        if is_heading and buffer:
            chunks.append("\n".join(buffer))
            buffer = [line]
        else:
            buffer.append(line)

    if buffer:
        chunks.append("\n".join(buffer))

    return chunks


def semantic_chunk_block(text: str, max_tokens: int = 1200, threshold: float = 0.65) -> List[str]:
    """
    Split a text block into semantic chunks using sentence embeddings.
    """
    from nltk.tokenize import sent_tokenize

    sentences = sent_tokenize(text)
    if len(sentences) <= 1:
        return [text]

    # Encode each sentence with CLIP embeddings
    embeddings = encode_text_clip(sentences)
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    chunks = []
    current = [sentences[0]]
    current_len = len(sentences[0])

    for i in range(1, len(sentences)):
        sim = float(np.dot(embeddings[i], embeddings[i - 1]))

        # Low similarity OR chunk too long → start new chunk
        if sim < threshold or current_len > max_tokens:
            chunks.append(" ".join(current))
            current = [sentences[i]]
            current_len = len(sentences[i])
        else:
            current.append(sentences[i])
            current_len += len(sentences[i])

    # final chunk
    chunks.append(" ".join(current))
    return chunks


def chunk_text(text: str) -> List[str]:
    """
    Hybrid chunking:
    1. Split using document structure (headings, sections).
    2. Within each block, perform semantic chunking.
    """
    structural_blocks = structural_split(text)
    final_chunks: List[str] = []

    for block in structural_blocks:
        block = block.strip()
        if not block:
            continue
        sem_chunks = semantic_chunk_block(block)
        final_chunks.extend(sem_chunks)

    return final_chunks


def items_to_chunks(items: List[Dict]) -> List[Dict]:
    """
    Convert per-page items into smaller chunks for embedding.
    Text → hybrid (structural + semantic).
    Image → OCR + caption stays together.
    """
    chunks: List[Dict] = []

    for it in items:
        if it["modality"] == "text":
            text_chunks = chunk_text(it["content"])
            for idx, ch in enumerate(text_chunks):
                chunks.append({
                    "id": f"{it['id']}_chunk{idx}",
                    "doc_id": it["doc_id"],
                    "modality": "text",
                    "content": ch,
                    "page_number": it["page_number"],
                    "section": it.get("section") or "",
                    "extra": {},  # not used for text in metadata; keep flat
                })
        elif it["modality"] == "image":
            combined = it["content"]
            caption = it["extra"].get("caption")
            if caption:
                combined = f"{caption}\n\n{combined}"

            chunks.append({
                "id": it["id"],
                "doc_id": it["doc_id"],
                "modality": "image",
                "content": combined,
                "page_number": it["page_number"],
                "section": it.get("section") or "",
                "extra": it.get("extra", {}),
            })

    return chunks



# Indexing (Chroma + CLIP)


def index_chunks(chunks: List[Dict]):
    """
    Index all chunks in unified multi-modal space.
    Text-like: CLIP text encoder.
    Images: CLIP image encoder, document string is OCR+caption.
    """
    if not chunks:
        return

    text_like = [c for c in chunks if c["modality"] != "image"]
    image_like = [c for c in chunks if c["modality"] == "image"]

    # Text-like chunks
    if text_like:
        texts = [c["content"] for c in text_like]
        text_embs = encode_text_clip(texts)
        collection.add(
            ids=[c["id"] for c in text_like],
            embeddings=text_embs.tolist(),
            documents=texts,
            metadatas=[
                {
                    "doc_id": str(c["doc_id"]),
                    "modality": str(c["modality"]),
                    "page_number": int(c["page_number"]),
                    "section": str(c.get("section") or ""),
                    # text chunks don't have image data
                    "image_path": "",
                    "caption": "",
                }
                for c in text_like
            ],
        )

    # Image chunks
    if image_like:
        image_paths = [c.get("extra", {}).get("image_path", "") for c in image_like]
        img_embs = encode_images_clip(image_paths)
        docs_for_llm = [c["content"] for c in image_like]  # OCR+caption
        collection.add(
            ids=[c["id"] for c in image_like],
            embeddings=img_embs.tolist(),
            documents=docs_for_llm,
            metadatas=[
                {
                    "doc_id": str(c["doc_id"]),
                    "modality": str(c["modality"]),
                    "page_number": int(c["page_number"]),
                    "section": str(c.get("section") or ""),
                    "image_path": c.get("extra", {}).get("image_path", ""),
                    "caption": c.get("extra", {}).get("caption", ""),
                }
                for c in image_like
            ],
        )

    print(f"Indexed {len(chunks)} chunks (text-like={len(text_like)}, image={len(image_like)})")

# -----------------------------
# Retrieval & RAG
# -----------------------------

def retrieve_chunks(query: str, doc_id: str, top_k: int = 8) -> List[Dict]:
    """Retrieve top-k multi-modal chunks for a given query and doc_id."""
    q_emb = encode_text_clip([query])
    results = collection.query(
        query_embeddings=q_emb.tolist(),
        n_results=top_k,
        where={"doc_id": doc_id},
    )

    if not results["ids"]:
        return []

    retrieved = []
    for i in range(len(results["ids"][0])):
        retrieved.append({
            "id": results["ids"][0][i],
            "content": results["documents"][0][i],
            "metadata": results["metadatas"][0][i],
            "distance": results["distances"][0][i] if "distances" in results else None,
        })
    return retrieved


def format_context_for_prompt(chunks: List[Dict]) -> str:
    """Produce LLM context string with explicit source tags."""
    blocks = []
    for idx, c in enumerate(chunks, start=1):
        m = c["metadata"]
        page = m.get("page_number", "?")
        section = m.get("section", "Unknown section")
        modality = m.get("modality", "text")
        tag = f"[SOURCE {idx} – page {page}, section: {section}, modality: {modality}]"
        content = c["content"].strip()
        if len(content) > 1200:
            content = content[:1200] + "... [truncated]"
        blocks.append(f"{tag}\n{content}\n")
    return "\n\n".join(blocks)


def build_rag_prompt(question: str, chunks: List[Dict]) -> str:
    context = format_context_for_prompt(chunks)
    prompt = f"""
You are a QA assistant for IMF-style financial and policy documents.

You are given context extracted from:
- Text paragraphs
- Tables (as text)
- Images and charts (represented via OCR text and captions)

Context:
{context}

Instructions:
- Answer the user's question ONLY using the context above.
- If the answer is not in the context, say you don't know.
- Always include citations using the exact tags like [SOURCE i – page X, section: Y, modality: Z].

Question: {question}

Answer:
"""
    # Trim if too long
    if len(prompt) > 6000:
        prompt = prompt[-6000:]
    return prompt


def call_llm(prompt: str, max_new_tokens: int = 256) -> str:
    tokenizer, gen_model = ModelRegistry.get_generator()
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048,
    ).to(DEVICE)

    with torch.no_grad():
        outputs = gen_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=4,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def rag_answer(question: str, doc_id: str, top_k: int = 8) -> Dict:
    chunks = retrieve_chunks(question, doc_id, top_k=top_k)
    if not chunks:
        return {
            "answer": "I couldn't find any relevant context for this document.",
            "sources": [],
        }
    prompt = build_rag_prompt(question, chunks)
    answer = call_llm(prompt)
    return {
        "answer": answer,
        "sources": chunks,
    }


# -----------------------------
# FastAPI models & app
# -----------------------------

class QueryRequest(BaseModel):
    doc_id: str
    question: str
    top_k: Optional[int] = 8


class QueryResponse(BaseModel):
    answer: str
    sources: List[Dict]


class IngestResponse(BaseModel):
    doc_id: str
    num_items: int
    num_chunks: int


app = FastAPI(title="Multi-Modal RAG over Documents")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.get("/")
async def root():
    return {"status": "ok", "message": "Multi-Modal RAG API is running."}


@app.post("/ingest", response_model=IngestResponse)
async def ingest_endpoint(
    file: UploadFile = File(...),
    doc_id: Optional[str] = Form(None),
):
    """Ingest a PDF into the RAG system."""
    if doc_id is None or not doc_id.strip():
        base = os.path.splitext(file.filename)[0] if file.filename else "doc"
        doc_id = f"{base}_{uuid.uuid4().hex[:8]}"

    file_bytes = await file.read()

    items = ingest_pdf_bytes(doc_id, file_bytes)
    chunks = items_to_chunks(items)
    index_chunks(chunks)

    return IngestResponse(
        doc_id=doc_id,
        num_items=len(items),
        num_chunks=len(chunks),
    )


@app.post("/query", response_model=QueryResponse)
async def query_endpoint(req: QueryRequest):
    """Ask a question about a previously ingested document."""
    result = rag_answer(req.question, req.doc_id, top_k=req.top_k or 8)
    return QueryResponse(
        answer=result["answer"],
        sources=result["sources"],
    )

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)




Writing main.py


In [11]:

from fastapi.responses import HTMLResponse




@app.get("/qa", response_class=HTMLResponse)
async def qa_frontend():
    """Simple HTML frontend for uploading a PDF and asking questions."""
    return """
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8" />
  <title>Multi-Modal RAG QA</title>
  <style>
    body {
      font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
      max-width: 900px;
      margin: 20px auto;
      padding: 0 16px;
      background: #f5f5f5;
    }
    h1 {
      text-align: center;
    }
    .card {
      background: white;
      padding: 16px 20px;
      margin-bottom: 20px;
      border-radius: 12px;
      box-shadow: 0 2px 6px rgba(0,0,0,0.08);
    }
    label {
      font-weight: 600;
      display: block;
      margin-bottom: 4px;
    }
    input[type="text"], textarea {
      width: 100%;
      padding: 8px 10px;
      border-radius: 8px;
      border: 1px solid #ccc;
      box-sizing: border-box;
      margin-bottom: 10px;
      font-family: inherit;
      font-size: 14px;
    }
    input[type="file"] {
      margin: 8px 0 12px;
    }
    button {
      padding: 8px 16px;
      border-radius: 999px;
      border: none;
      cursor: pointer;
      font-weight: 600;
      font-size: 14px;
      margin-top: 4px;
    }
    button.primary {
      background: #2563eb;
      color: white;
    }
    button.secondary {
      background: #e5e7eb;
      color: #111827;
    }
    .row {
      display: flex;
      gap: 8px;
      align-items: center;
    }
    .row input[type="text"] {
      flex: 1;
      margin-bottom: 0;
    }
    .pill {
      display: inline-block;
      padding: 2px 8px;
      font-size: 12px;
      border-radius: 999px;
      background: #e5e7eb;
      margin-right: 4px;
    }
    pre {
      white-space: pre-wrap;
      background: #111827;
      color: #e5e7eb;
      padding: 12px;
      border-radius: 8px;
      font-size: 13px;
      max-height: 300px;
      overflow: auto;
    }
    #status {
      margin-top: 6px;
      font-size: 13px;
      color: #4b5563;
    }
    .source {
      background: #f9fafb;
      padding: 8px 10px;
      border-radius: 8px;
      margin-bottom: 6px;
      font-size: 13px;
    }
    .source-header {
      font-weight: 600;
      margin-bottom: 4px;
    }
  </style>
</head>
<body>
  <h1>📄 Multi-Modal RAG QA</h1>

  <!-- Upload / ingest card -->
  <div class="card">
    <h2>1. Upload Document</h2>
    <label for="pdfFile">PDF file</label>
    <input type="file" id="pdfFile" accept="application/pdf" />

    <label for="docIdInput">Document ID (optional)</label>
    <div class="row">
      <input type="text" id="docIdInput" placeholder="Leave empty to auto-generate" />
      <button class="secondary" type="button" onclick="clearDocId()">Clear</button>
    </div>

    <button class="primary" type="button" onclick="uploadPdf()">Ingest PDF</button>
    <div id="status"></div>
    <div id="currentDoc"></div>
  </div>

  <!-- QA card -->
  <div class="card">
    <h2>2. Ask a Question</h2>
    <label for="questionInput">Question</label>
    <textarea id="questionInput" rows="3" placeholder="Example: What is the GDP growth outlook for next year?"></textarea>

    <div class="row">
      <label for="topKInput" style="margin: 0;">Top-k chunks:</label>
      <input type="text" id="topKInput" value="8" style="max-width: 80px;" />
      <button class="primary" type="button" onclick="askQuestion()">Ask</button>
    </div>

    <div id="answerBox" style="margin-top: 12px;"></div>
    <div id="sourcesBox" style="margin-top: 10px;"></div>
  </div>

  <script>
    const baseUrl = window.location.origin;  // same host as FastAPI

    function setStatus(msg) {
      document.getElementById("status").innerText = msg || "";
    }

    function setCurrentDoc(docId) {
      if (!docId) {
        document.getElementById("currentDoc").innerHTML = "";
        return;
      }
      document.getElementById("currentDoc").innerHTML =
        '<span class="pill">Current doc_id:</span> <code>' + docId + '</code>';
    }

    function clearDocId() {
      document.getElementById("docIdInput").value = "";
      setCurrentDoc("");
    }

    async function uploadPdf() {
      const fileInput = document.getElementById("pdfFile");
      const docIdInput = document.getElementById("docIdInput");

      if (fileInput.files.length === 0) {
        alert("Please select a PDF file first.");
        return;
      }

      const file = fileInput.files[0];
      const formData = new FormData();
      formData.append("file", file);
      if (docIdInput.value.trim() !== "") {
        formData.append("doc_id", docIdInput.value.trim());
      }

      setStatus("Uploading and ingesting PDF...");

      try {
        const resp = await fetch(baseUrl + "/ingest", {
          method: "POST",
          body: formData
        });

        if (!resp.ok) {
          const text = await resp.text();
          console.error("Ingest error:", text);
          setStatus("Error during ingest: " + resp.status + " " + resp.statusText);
          return;
        }

        const data = await resp.json();
        console.log("Ingest response:", data);
        setStatus("Ingestion complete. Chunks indexed: " + data.num_chunks);
        docIdInput.value = data.doc_id;
        setCurrentDoc(data.doc_id);
      } catch (err) {
        console.error(err);
        setStatus("Error during ingest: " + err);
      }
    }

    async function askQuestion() {
      const docId = document.getElementById("docIdInput").value.trim();
      const question = document.getElementById("questionInput").value.trim();
      const topKStr = document.getElementById("topKInput").value.trim();

      if (!docId) {
        alert("Please upload a document first (or specify doc_id).");
        return;
      }
      if (!question) {
        alert("Please enter a question.");
        return;
      }

      const topK = parseInt(topKStr || "8", 10);

      setStatus("Asking question...");
      document.getElementById("answerBox").innerHTML = "";
      document.getElementById("sourcesBox").innerHTML = "";

      try {
        const resp = await fetch(baseUrl + "/query", {
          method: "POST",
          headers: { "Content-Type": "application/json" },
          body: JSON.stringify({
            doc_id: docId,
            question: question,
            top_k: topK
          }),
        });

        if (!resp.ok) {
          const text = await resp.text();
          console.error("Query error:", text);
          setStatus("Error during query: " + resp.status + " " + resp.statusText);
          return;
        }

        const data = await resp.json();
        console.log("Query response:", data);
        setStatus("Done.");

        // Show answer
        document.getElementById("answerBox").innerHTML =
          "<h3>Answer</h3><pre>" + (data.answer || "[no answer]") + "</pre>";

        // Show sources
        const sourcesDiv = document.getElementById("sourcesBox");
        sourcesDiv.innerHTML = "<h3>Sources</h3>";
        if (!data.sources || data.sources.length === 0) {
          sourcesDiv.innerHTML += "<p>No sources returned.</p>";
        } else {
          data.sources.forEach((s, idx) => {
            const md = s.metadata || {};
            const src = document.createElement("div");
            src.className = "source";
            src.innerHTML =
              '<div class="source-header">SOURCE ' + (idx + 1) +
              " – page " + (md.page_number ?? "?") +
              ', modality: ' + (md.modality ?? "?") +
              ', section: ' + (md.section || "") +
              "</div>" +
              "<div>" + (s.content ? s.content.substring(0, 400) : "") +
              (s.content && s.content.length > 400 ? "..." : "") +
              "</div>";
            sourcesDiv.appendChild(src);
          });
        }

      } catch (err) {
        console.error(err);
        setStatus("Error during query: " + err);
      }
    }
  </script>
</body>
</html>
    """




#FASTAPI+ngrok Building

In [12]:
import nest_asyncio, threading, uvicorn
from pyngrok import ngrok
from main import app  # our FastAPI app

nest_asyncio.apply()

# Use the token you already stored in NGROK_AUTH_TOKEN
from os import environ
token = environ.get("NGROK_AUTH_TOKEN")
if not token:
    raise RuntimeError("NGROK_AUTH_TOKEN env var not set. Run the getpass setup cell first.")

ngrok.set_auth_token(token)

# Open public tunnel
public_tunnel = ngrok.connect(8000, "http")
public_url = public_tunnel.public_url
print("Public URL:", public_url)

# Start FastAPI server in background thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()


Public URL: https://heavily-tubbier-tajuana.ngrok-free.dev


INFO:     Started server process [1212]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
